<a href="https://colab.research.google.com/github/singh728om/Air-Pollution-Monitoring-System/blob/master/Indigo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split

# Step 1: Load and explore the dataset
dataset = load_dataset('toughdata/quora-question-answer-dataset')
print(dataset)

# Check for the splits in the dataset
if 'validation' not in dataset:
    # Assuming the train split needs to be split into train and validation
    train_valid = dataset['train'].train_test_split(test_size=0.1)  # 10% of train data for validation
    dataset = DatasetDict({
        'train': train_valid['train'],
        'validation': train_valid['test']
    })
print(dataset)

# Assuming the correct columns are 'question' and 'answer_text'
question_col = 'question'  # Replace with actual column name if different
context_col = 'answer'  # Replace with actual column name if different

# Step 2: Preprocess the data
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

def preprocess_data(examples):
    questions = examples[question_col]
    contexts = examples[context_col]
    inputs = tokenizer(questions, contexts, truncation=True, padding=True)
    return inputs

tokenized_dataset = dataset.map(preprocess_data, batched=True)

# Step 3: Model selection
model = AutoModelForQuestionAnswering.from_pretrained('bert-base-uncased')

# Step 4: Training
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation']
)

# Train the model
trainer.train()

# Step 5: Evaluation
results = trainer.evaluate()
print(f"Evaluation results: {results}")

# Step 6: Save the model
model.save_pretrained('./qa_model')
tokenizer.save_pretrained('./qa_model')


DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 56402
    })
})
DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 50761
    })
    validation: Dataset({
        features: ['question', 'answer'],
        num_rows: 5641
    })
})


Map:   0%|          | 0/50761 [00:00<?, ? examples/s]

Map:   0%|          | 0/5641 [00:00<?, ? examples/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
